<a href="https://colab.research.google.com/github/Sudhan22z265/Machine_learning/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##**Project Twitter US Airline Sentiment Analysis**


#Reading Dataset

In [88]:
import pandas as pd

In [89]:
data = pd.read_csv('training_twitter_x_y_train.csv')

In [90]:
df = data.copy()

In [91]:
df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


#Storing airline sentiment in df_cat

*   cat -> category




In [92]:
df_cat = df['airline_sentiment']

#Storing airline twitter text in df_msg


In [93]:
df_msg = df['text']

In [94]:
df_cat.head()

0    negative
1    positive
2    positive
3    negative
4    negative
Name: airline_sentiment, dtype: object

In [95]:
df_msg.head()

0    @SouthwestAir I am scheduled for the morning, ...
1    @SouthwestAir seeing your workers time in and ...
2    @united Flew ORD to Miami and back and  had gr...
3       @SouthwestAir @dultch97 that's horse radish 😤🐴
4    @united so our flight into ORD was delayed bec...
Name: text, dtype: object

#Converting dataFrame to list


*   storing df_cat as cat(list)
*   storing df_msg as msg(list)



In [96]:
cat = list(df_cat)

In [ ]:
msg = list(df_msg)
msg

In [98]:
msg[0]

'@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'

#Tokenizing words and stored it as array of tuples
#Each tuple contains list of tokenized words and its sentiment

In [99]:
import nltk
from nltk.tokenize import word_tokenize


documents=[]
for i in range(len(msg)):
  new_msg=()
  tokened_word = word_tokenize(msg[i])
  new_msg += (tokened_word,)
  new_msg+=(cat[i],)
  documents.append(new_msg)




In [ ]:
documents[:5]

#Shuffling the datas

In [101]:
import random
random.shuffle(documents)

In [ ]:
documents[:5]

In [103]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()




*   **Removing Stopwords(i.e, most commonly used words)**
*   **Removing punctuations**

*   **Making all words as Root words using Lemmatization**







In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

stopwords = stopwords.words('english')

stopwords = stopwords + list(string.punctuation)
stopwords

In [105]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('N'):
    return wordnet.NOUN
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [106]:
from nltk import pos_tag
def clean_words(words):
  output_words = []
  for w in words:
    if w.lower() not in stopwords:
      pos = pos_tag([w])
      clean_word = lemmatizer.lemmatize(w,pos =get_simple_pos(pos[0][1]))
      output_words.append(clean_word.lower())
  return output_words

#Cleaning Words

In [107]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
documents = [(clean_words(i),j) for i,j in documents]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [108]:
len(documents)

10980

#Splitting training and testing data

In [109]:
documents_training = documents[0:9000]
documents_testing = documents[9000:]

In [110]:
allwords = []
for doc in documents:
  allwords+=doc[0]

#Getting Frequency Distribution

In [111]:
import nltk
freq = nltk.FreqDist(allwords)

In [112]:
common = freq.most_common(3000)
features =  [i[0] for i in common]

#Choosing features as most common words

In [ ]:
features[0:20]

In [114]:
def get_feature(words):
  current_features = {}
  words_set = set(words)
  for w in features:
    current_features[w] = w in words_set
  return current_features

In [115]:
train_data = [(get_feature(i),j) for i,j in documents_training]
test_data = [(get_feature(i),j) for i,j in documents_testing]

In [ ]:
train_data[0:2]

#Training NaiveBayes Classifier

In [116]:
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)


#Checking accuracy for testing data

In [117]:
nltk.classify.accuracy(classifier,test_data)

0.7722222222222223

#Getting most informative features

In [119]:
classifier.show_most_informative_features(10)

Most Informative Features
                   kudos = True           positi : negati =     44.4 : 1.0
               fantastic = True           positi : negati =     29.2 : 1.0
                favorite = True           positi : negati =     29.2 : 1.0
             outstanding = True           positi : negati =     29.2 : 1.0
                   march = True           neutra : negati =     24.7 : 1.0
                 awesome = True           positi : negati =     23.5 : 1.0
                   thank = True           positi : negati =     22.7 : 1.0
                    hold = True           negati : positi =     21.3 : 1.0
                   great = True           positi : neutra =     19.8 : 1.0
                   sweet = True           positi : negati =     19.0 : 1.0
